In [1]:
import openmeteo_requests
import numpy as np
import requests_cache
import pandas as pd
from retry_requests import retry
import requests
import time 

In [2]:
commune_de_france=requests.get('https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25')
with open("communes_de_france.csv", "wb") as f:
    f.write(commune_de_france.content)

communes_de_france = pd.read_csv('communes_de_france.csv')
communes_de_france["latitude"].dropna(inplace=True)
communes_de_france["longitude"].dropna(inplace=True)

communes_de_france_partie1 = communes_de_france.iloc[:5000]
communes_de_france_partie1bis = communes_de_france.iloc[5001:10000]
communes_de_france_partie2 = communes_de_france.iloc[100001:20000] 
communes_de_france_partie3 = communes_de_france.iloc[20001:30000] 
communes_de_france_partie4 = communes_de_france.iloc[30001:] 


températures prédites en 2023 pour les communes de France

In [ ]:
data_frame_storage_past_forecast=pd.DataFrame(columns=['date','temperature_2m', 'latitude', "longitude"])
		# Setup the Open-Meteo API client with cache and retry on error
for i in range(len(communes_de_france_partie1)):
				cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
				retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
				openmeteo = openmeteo_requests.Client(session = retry_session)

				# Make sure all required weather variables are listed here
				# The order of variables in hourly or daily is important to assign them correctly below
				url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
				params = {
					"latitude": communes_de_france_partie1["latitude"].iloc[i],
					"longitude": communes_de_france_partie1["longitude"].iloc[i],
					"start_date": "2023-01-01",
					"end_date": "2023-12-31",
					"hourly": "temperature_2m",
					"models": "meteofrance_seamless"
				}
				responses = openmeteo.weather_api(url, params=params)
								# Process first location. Add a for-loop for multiple locations or weather models
				response = responses[0]
				# Process hourly data. The order of variables needs to be the same as requested.
				hourly = response.Hourly()
				hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

				hourly_data = {"date": pd.date_range(
					start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
					end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
					freq = pd.Timedelta(seconds = hourly.Interval()),
					inclusive = "left"
				)}
				hourly_data["temperature_2m"] = hourly_temperature_2m


				hourly_dataframe = pd.DataFrame(data = hourly_data)
				hourly_dataframe.dropna(inplace=True)
				hourly_dataframe["latitude"] = [communes_de_france_partie1["latitude"].iloc[i] for k in range(len(hourly_dataframe))]
				hourly_dataframe["longitude"] = [communes_de_france_partie1["longitude"].iloc[i] for k in range(len(hourly_dataframe))]
				data_frame_storage_past_forecast = pd.concat([data_frame_storage_past_forecast, hourly_dataframe], axis=0)
				time.sleep(1)


nom='france_communes_hourly_past_predicted_temperature_2m_2023.csv'
data_frame_storage_past_forecast.to_csv(nom)